Para garantir uma organização e precisão adequadas, o tratamento dos dados foi realizado em etapas. A seguir, apresento o processo detalhado:

Passo 1: Tratamento da Tabela PIP_municipio
1.	Importação dos Dados: Os dados foram importados da aba PIP_municipio.
2.	Tratamento dos Dados: Realizei os tratamentos necessários para adequar os dados.
Passo 2: Tratamento das Tabelas UF_regiao e De_para_UF
1.	Importação dos Dados: Importei as abas UF_regiao e De_para_UF.
2.	Tratamento dos Dados: Efetuei os tratamentos necessários em cada tabela.
3.	Merge das Tabelas: Realizei um merge entre as tabelas para obter as colunas Estado, Região e UF.
Passo 3: Integração com a Tabela PIP_municipio
1.	Merge com PIP_municipio: Combinei a tabela resultante do Passo 2 com a tabela PIP_municipio já tratada.
2.	Agrupamento por Região: Agrupei os dados por Região e somei os valores do PIP para cada agrupamento, além da soma dos valores totais.
Passo 4: Tratamento da Tabela População_Estado
1.	Importação dos Dados: Importei os dados da aba População_Estado.
2.	Tratamento dos Dados: Realizei os tratamentos necessários para esta tabela.
3.	Merge com UF_regiao: Utilizei o DataFrame já tratado UF_regiao para efetuar um merge, adicionando as colunas de regiões ao DataFrame da população.
4.	Agrupamento e Soma: Agrupei os dados por Região e somei os valores da população para cada agrupamento, além da soma dos valores totais.
Passo 5: Cálculo do PIP per Capita
1.	Merge dos DataFrames: Combinei os DataFrames de PIP e População para integrar os dados.
2.	Cálculo do PIP per Capita: Calculei o PIP per capita dividindo o valor total do PIP pela população de cada região.
3.	Seleção de Colunas: Filtrei o DataFrame para manter apenas as colunas solicitadas pelo teste.
Passo 6: Exportação dos Dados
1.	Exportação para Excel: Exportei o DataFrame final para um arquivo Excel.
2.	Remoção do Índice: Excluí o índice durante a exportação para garantir que o arquivo final estivesse conforme solicitado.

Mais detalhes sobre cada etapa podem ser encontrados nos comentários do código.

In [71]:
import pandas as pd

municipio = pd.read_excel("Base_Teste_Original.xlsx", sheet_name="PIB_municipio")
municipio.info()
municipio.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Cod_Identificacao  5570 non-null   int64 
 1   Municipio          5570 non-null   object
 2   PIB                5570 non-null   object
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


,Cod_Identificacao,Municipio,PIB
0,1100015,Alta Floresta D'Oeste (RO),262077
1,1100023,Ariquemes (RO),1364694
2,1100031,Cabixi (RO),69611
3,1100049,Cacoal (RO),1186494
4,1100056,Cerejeiras (RO),222021


In [72]:
# transformando a coluna PIP en int e tratando os nulos com 0.
municipio["PIB"] = pd.to_numeric(municipio["PIB"], errors="coerce").fillna(0)
municipio["PIB"] = municipio["PIB"].astype(int)
municipio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Cod_Identificacao  5570 non-null   int64 
 1   Municipio          5570 non-null   object
 2   PIB                5570 non-null   int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 108.9+ KB


In [73]:
#Troquei o nome da coluna original, criei duas novas colunas, separei usando um delimitador, tratei o excedente e verifiquei valores duplicados.

municipio.rename(columns={"Municipio": "Municipio_Sigla" }, inplace =True)
municipio[['Municipio', 'UF']] = municipio['Municipio_Sigla'].str.split("(", expand=True)
municipio["UF"] = municipio["UF"].str.replace(")", "")
muni_dupli = municipio["Municipio_Sigla"].duplicated().any()
muni_dupli, municipio


(False,
       Cod_Identificacao             Municipio_Sigla        PIB  \
 0               1100015  Alta Floresta D'Oeste (RO)     262077   
 1               1100023              Ariquemes (RO)    1364694   
 2               1100031                 Cabixi (RO)      69611   
 3               1100049                 Cacoal (RO)    1186494   
 4               1100056             Cerejeiras (RO)     222021   
 ...                 ...                         ...        ...   
 5565            5222005             Vianópolis (GO)     178699   
 5566            5222054         Vicentinópolis (GO)     133899   
 5567            5222203               Vila Boa (GO)      71676   
 5568            5222302          Vila Propício (GO)      92063   
 5569            5300108               Brasília (DF)  144174102   
 
                    Municipio  UF  
 0     Alta Floresta D'Oeste   RO  
 1                 Ariquemes   RO  
 2                    Cabixi   RO  
 3                    Cacoal   RO  
 4    

In [74]:
regiao = pd.read_excel("Base_Teste_Original.xlsx", sheet_name="UF_Regiao")
de_para = pd.read_excel("Base_Teste_Original.xlsx", sheet_name="De_para_UF")
regiao.info(), de_para.info()
regiao.head(5), de_para.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Estado  27 non-null     object
 1   Regiao  27 non-null     object
dtypes: object(2)
memory usage: 564.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Granularidade      28 non-null     object
 1   UF                 28 non-null     object
 2   Cod_Identificacao  28 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 804.0+ bytes


(     Estado Regiao
 0  Rondonia      N
 1      Acre      N
 2  Amazonas      N
 3   Roraima      N
 4      Para      N,
   Granularidade  UF  Cod_Identificacao
 0          Acre  AC                 12
 1       Alagoas  AL                 27
 2         Amapa  AP                 16
 3      Amazonas  AM                 13
 4         Bahia  BA                 29)

In [75]:
#Mapeei as regiões com os valores solicitados de saída e tratei valores duplicados.

UF_map = {
    'CO': 'Centro-Oeste',
    'N': 'Norte',
    'NE': 'Nordeste',
    'S': 'Sul',
    'SE': 'Sudeste'
}
regiao["Regiao"] = regiao["Regiao"].replace(UF_map)

de_para =de_para.drop_duplicates()
de_para.info()
de_para

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Granularidade      27 non-null     object
 1   UF                 27 non-null     object
 2   Cod_Identificacao  27 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 864.0+ bytes


,Granularidade,UF,Cod_Identificacao
0,Acre,AC,12
1,Alagoas,AL,27
2,Amapa,AP,16
3,Amazonas,AM,13
4,Bahia,BA,29
5,Ceara,CE,23
6,Distrito Federal,DF,53
7,Espirito Santo,ES,32
8,Goias,GO,52
9,Maranhao,MA,21


In [76]:
# Validando valores da coluna UF, há 1 valor duplicado já que são 26 Estados + Distrito Federal.
de_para = de_para.drop("Cod_Identificacao", axis=1)
de_para = de_para.rename(columns={"Granularidade": "Estado"})
de_para["UF"].nunique()


26

In [77]:
# Identificando valores duplicados na coluna UF.
dupli_de_para = de_para[de_para.duplicated(subset=["UF"],keep=False)]
dupli_de_para

,Estado,UF
10,Mato Grosso,MG
12,Minas Gerais,MG


In [78]:
# Alteração dos dados inconsistentes.
de_para.at[10, "UF"] = "MT"
linha_10_12 = de_para.loc[10:12]
linha_10_12

,Estado,UF
10,Mato Grosso,MT
11,Mato Grosso do Sul,MS
12,Minas Gerais,MG


In [79]:
# Join entre as tabelas, objetivo: obter as colunas região, UF e Estado na mesma tabela.
regiao_uf = pd.merge(regiao, de_para, how="outer", on="Estado")
regiao_uf 

,Estado,Regiao,UF
0,Acre,Norte,AC
1,Alagoas,Nordeste,AL
2,Amapa,Norte,AP
3,Amazonas,Norte,AM
4,Bahia,Nordeste,BA
5,Ceara,Nordeste,CE
6,Distrito Federal,Centro-Oeste,DF
7,Espirito Santo,Sudeste,ES
8,Goias,Centro-Oeste,GO
9,Maranhao,Nordeste,MA


In [80]:
#Join entre as tabelas, objetivo: obter o total do PIP das regiões.

reg_muni = pd.merge(municipio, regiao_uf, how="left", on="UF")
reg_pib = reg_muni[["Regiao", "PIB"]]
agru_pib = reg_pib.groupby("Regiao").sum().reset_index()
soma_pib = agru_pib["PIB"].sum()
agru_pib.loc[len(agru_pib)] = ["Total", soma_pib]
agru_pib

,Regiao,PIB
0,Centro-Oeste,354815818
1,Nordeste,522769299
2,Norte,207093641
3,Sudeste,2180987793
4,Sul,620180421
5,Total,3885846972


In [98]:
#Importação da tabela e verificação dos dados na coluna Granularidade.

pop_estado = pd.read_excel("Base_Teste_Original.xlsx", sheet_name="Populacao_Estado")
pop_estado["Granularidade"].unique()


array(['Brasil', 'Rondonia', 'Acre', 'Amazonas', 'Roraima', 'Para',
       'Amapa', 'Tocantins', 'Maranhao', 'Piaui', 'Ceara',
       'Rio Grande do Norte', 'Paraiba', 'Pernambuco', 'Alagoas',
       'Sergipe', 'Bahia', 'Minas Gerais', 'Espirito Santo',
       'Rio de Janeiro', 'Sao Paulo', 'Parana', 'Santa Catarina',
       'Rio Grande do Sul', 'Mato Grosso do Sul', 'Mato Grosso', 'Goias',
       'Distrito Federal'], dtype=object)

In [99]:
#Verificação da coluna
pop_estado["fx_idade"].unique()

array(['Total', '0 a 4 anos', '5 a 9 anos', '10 a 14 anos',
       '15 a 19 anos', '20 a 24 anos', '25 a 29 anos', '30 a 34 anos',
       '35 a 39 anos', '40 a 44 anos', '45 a 49 anos', '50 a 54 anos',
       '55 a 59 anos', '60 a 64 anos', '65 a 69 anos', '70 a 74 anos',
       '75 a 79 anos', '80 a 84 anos', '85 a 89 anos', '90 a 94 anos',
       '95 a 99 anos', '100 anos ou mais'], dtype=object)

In [83]:
#Verificando se o total da população de cada estado é igual a soma dos totais menos a população total do Brasil.

total_estado = pop_estado[(pop_estado["fx_idade"] == "Total") & (pop_estado["Granularidade"] != "Brasil")]
pop_estado = pop_estado[(pop_estado["fx_idade"] != "Total") & (pop_estado["Granularidade"] != "Brasil")]

total_estado["Populacao"].sum(),pop_estado["Populacao"].sum()

(190755799, 190755797)

In [84]:
"""
Após verificar as discrepâncias entre os valores totais e as somas por estado, decidi utilizar os dados detalhados por estado para garantir maior precisão nos cálculos.
Realizei a junção das tabelas e agrupei os dados por região, somando os valores da população para cada uma e somei os valores de todas as regiões e adicionei o total 
geral na última linha do DataFrame.

"""
pop_estado = pop_estado.drop(columns="fx_idade")
pop_estado = pop_estado.groupby("Granularidade").sum().reset_index()
pop_estado = pop_estado.rename(columns={"Granularidade" : "Estado"})
pop = pd.merge(pop_estado, regiao, on="Estado", how="left" )
agru_pop = pop.drop("Estado", axis=1).groupby("Regiao").sum().reset_index()
soma_pop = pop_estado["Populacao"].sum()
agru_pop.loc[len("Populacao")] = ["Total", soma_pop]
agru_pop

,Regiao,Populacao
0,Centro-Oeste,14058093
1,Nordeste,53081952
2,Norte,15864452
3,Sudeste,80364408
4,Sul,27386892
9,Total,190755797


In [93]:
# Join entre os dois dataframes para obter os valores de população e PIP, efetuei o cálculo para ter o PIP per capita e organizei de forma crescente.  

pip_per = pd.merge(agru_pop, agru_pib, on= "Regiao", how="left")
pip_per["PIP_per_capita"] = pip_per["PIB"]/pip_per["Populacao"]
pip_per.sort_values(by="PIP_per_capita", inplace=True)
pip_per

,Regiao,Populacao,PIB,PIP_per_capita
1,Nordeste,53081952,522769299,9.848344
2,Norte,15864452,207093641,13.053942
5,Total,190755797,3885846972,20.370794
4,Sul,27386892,620180421,22.645155
0,Centro-Oeste,14058093,354815818,25.239257
3,Sudeste,80364408,2180987793,27.138728


In [94]:
#Arredondei os valores do PIP per capita e deixei somente as colunas desejadas. Está comentado a tranformação de "." para ",", não sabia se era necessário.

pip_per.loc[:, "PIP_per_capita"] = pip_per["PIP_per_capita"].round(1)
pip_per=pip_per[["Regiao", "PIP_per_capita"]]
#pip_per.loc[:, "PIP_per_capita"] = pip_per["PIP_per_capita"].astype(str)
#pip_per.loc[:, "PIP_per_capita"] = pip_per["PIP_per_capita"].str.replace(".", ",")
pip_per

,Regiao,PIP_per_capita
1,Nordeste,9.8
2,Norte,13.1
5,Total,20.4
4,Sul,22.6
0,Centro-Oeste,25.2
3,Sudeste,27.1


In [96]:
pip_per.to_excel("Base_Final.xlsx", index=False)